In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import geopandas as gpd


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Spatial Polygon 

In [ ]:
import geopandas as gpd

geo_df = gpd.read_file("/kaggle/input/map-data/INDONESIA_PROP.shp")


In [ ]:
leg_kwds = {'title':'NAMA WILAYAH', 'loc':'upper left', 'bbox_to_anchor':(1,1.05), 'ncol':1}
geo_df.plot(column='Propinsi', cmap='Set1', legend=False, legend_kwds = leg_kwds, figsize = (30,30))
plt.title('TITIK GEMPA')
plt.show()

In [ ]:
# get data 
geo_ = gpd.read_file("/kaggle/input/map-data/INDONESIA_PROP.shp")
# data can get 
filter_sumatra_jawa = [ "SUMATERA UTARA", "SUMATERA BARAT","JAWA TENGAH","PROBANTEN","DI. ACEH","JAWA TIMUR","JAWA BARAT" ,"LAMPUNG" ,"DKI JAKARTA","RIAU", "JAMBI","BANGKA BELITUNG" ,"SUMATERA SELATAN","DAERAH ISTIMEWA YOGYAKARTA"]

ori_len = len(geo_)
geo_['Wilayah'] = list(range(len(geo_)))
# filter data 
for i in range(ori_len):
    if geo_['Propinsi'][i] in filter_sumatra_jawa :
        geo_['Wilayah'][i] = 1
    else:
        geo_['Wilayah'][i] = 0
geo_ = geo_[geo_['Wilayah'] == 1]
geo_.head()

In [ ]:
leg_kwds = {'title':'NAMA WILAYAH', 'loc':'upper left','bbox_to_anchor':(1,1.05), 'ncol':1}
geo_.plot(column='Propinsi', cmap='Set1', legend=True, legend_kwds = leg_kwds, figsize = (20,50))
plt.title('TITIK GEMPA')
plt.show()

# Spatial Point and Spatial Polygon

## Praprosesing Data 

data di ambil dari data inatews , pada proses praprosesing ini kita akan membersikan data merubah format dan lain-lian

In [ ]:
train_09 = pd.read_csv('/kaggle/input/data-gempa/2009.csv')
train_10 = pd.read_csv('/kaggle/input/data-gempa/2010.csv')
train_11 = pd.read_csv('/kaggle/input/data-gempa/2011.csv')
train_12 = pd.read_csv('/kaggle/input/data-gempa/2012.csv')
train_13 = pd.read_csv('/kaggle/input/data-gempa/2013.csv')
train_14 = pd.read_csv('/kaggle/input/data-gempa/2014.csv')
train_15 = pd.read_csv('/kaggle/input/data-gempa/2015.csv')
train_16 = pd.read_csv('/kaggle/input/data-gempa/2016.csv')
train_17 = pd.read_csv('/kaggle/input/data-gempa/2017.csv')
train_18 = pd.read_csv('/kaggle/input/data-gempa/2018.csv')
train_all = [train_09,train_10,train_11,train_12,train_13,train_14,train_15,train_16,train_17,train_18 ]

# me replace name stiap data
for i in train_all:
    i.columns = ['index', 'Date', 'Time', 'Latitude', 'Longitude','Depth',
              'Magnitude', 'TypeMag', 'smaj', 'smin', 'az', 'rms', 'cPhase',
               'Region']
    i.drop(0,axis=0,inplace=True)
    
# menyatukan data mendjadi 1 
train = pd.concat(train_all)

# mengsplit data dan membersihkan nilai menjadi number
train["Longitude"] = train.Longitude.apply(lambda x : x.split()[0])
train["Depth"] = train.Depth.apply(lambda x : x.split()[0])
train["Magnitude"] = train.Magnitude.apply(lambda x : x.split()[0])
train["Time"] = train.Time.apply(lambda x : x.split()[0])

# meng ekstrak data 
train["year"] = train.Date.apply(lambda x : x.split('/')[2])
train["date"] = train.Date.apply(lambda x : x.split('/')[1])
train["month"] = train.Date.apply(lambda x : x.split('/')[0])
train["hour"] = train.Time.apply(lambda x : x.split(':')[0])


train["Date"] = train.Date.apply(lambda x : x.split()[0])
train['Date'] = train[['Date', 'Time']].apply(lambda x : ' '.join(x), axis = 1)
train['Date'] = pd.to_datetime(train['Date'])

train.index = train['Date']

In [ ]:
train_len = len(train)
for i in range(train_len):
    if train['Latitude'][i].split()[1] == 'LS':
        train['Latitude'][i] = float(train['Latitude'][i].split()[0]) * -1
    else:
        train['Latitude'][i] = train['Latitude'][i].split()[0]

#convert type data ke float
columns = ['Latitude','Longitude', 'Depth', 'Magnitude']
for var in columns:
    train[var] = train[var].astype("float64")
    



In [ ]:
train.head()

In [ ]:
# Filter indonesia
train = train.loc[((train['Latitude'] >= -12.5) & (train['Latitude'] <= 7) &  (train['Longitude'] >= 94.31644) & (train['Longitude'] <= 142.71813) )]

In [ ]:
train.head()

In [ ]:
leg_kwds = {'title':'NAMA WILAYAH', 'loc':'upper left', 'bbox_to_anchor':(1,1.05), 'ncol':1}
geo_df.plot(column='Propinsi', cmap='Set1', legend=True, legend_kwds = leg_kwds, figsize = (30,30))
plt.title('TITIK GEMPA')

plt.scatter(y= train.Latitude, x = train.Longitude , c = train.Magnitude, alpha=0.8);
plt.show()

In [ ]:
train5 = train[train['Magnitude'] >= 5.5]

In [ ]:
leg_kwds = {'title':'NAMA WILAYAH', 'loc':'upper left', 'bbox_to_anchor':(1,1.05), 'ncol':1}
geo_df.plot(column='Propinsi', cmap='Set1', legend=True, legend_kwds = leg_kwds, figsize = (30,30))
plt.title('TITIK GEMPA')

plt.scatter(y= train5.Latitude, x = train5.Longitude , c = train5.Magnitude, alpha=0.8);
plt.show()

# Analysis Network Station seismik with earthquake

## Menentukan Lokasi Stasiun gempa,
dengan mengasumsikan bahwa gempa yang memiliki jarak (smin) kurang dari 0.3 (300) , disitulah daerah sekitar yang dekat dengan stasiun gempa
setelah itu kita melakukan clustering menggunakan k-means , untuk menentukan titik yang benar-benar tepat atau mengambil sentorid dari persebaran tersebut

In [ ]:
train_loc = train.copy()

train_loc = train_loc[train_loc.smin < 0.3]

leg_kwds = {'title':'NAMA WILAYAH', 'loc':'upper left', 'bbox_to_anchor':(1,1.05), 'ncol':1}
geo_df.plot(column='Propinsi', cmap='Set1', legend=True, legend_kwds = leg_kwds, figsize = (30 ,30))
plt.title('TITIK GEMPA')

plt.scatter(y= train_loc.Latitude, x = train_loc.Longitude, alpha=0.8);
plt.show()

## melakukan k-means clustering untuk menentukan 

In [ ]:
# 164 - 12 = 152
from sklearn.cluster import KMeans

train_clus = train_loc[['Latitude', 'Longitude']]
num_clusters = 152

km = KMeans(n_clusters=num_clusters, random_state = 1)
km.fit(train_clus)
clusters = km.labels_.tolist()

train_clus['Clustering'] = clusters


loc_statiun = train_clus.groupby('Clustering').mean()
loc_statiun = loc_statiun.reset_index()

In [ ]:
leg_kwds = {'title':'NAMA WILAYAH', 'loc':'upper left', 'bbox_to_anchor':(1,1.05), 'ncol':1}
geo_df.plot(column='Propinsi', cmap='Set1', legend=False, legend_kwds = leg_kwds, figsize = (30,30))
plt.title('Lokasi Statiun Gempa')

plt.scatter(y= loc_statiun.Latitude, x = loc_statiun.Longitude, marker='^', alpha=1, c = '#1f77b4');
plt.show()

In [ ]:
train.head()

## Menghitung dan menentukan stasiun terdekat dari setiap gempa 

In [ ]:
statiun_coords = {}

for dat in loc_statiun.iterrows():
    row = dat[1]
    
    statiun_coords[int(row['Clustering'])] = (float(row['Latitude']), float(row['Longitude']))

In [ ]:
#from pandas.tools.plotting import scatter_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from geopy.distance import vincenty

import gc

In [ ]:
def closest_point(location, location_dict):
    """ take a tuple of latitude and longitude and 
        compare to a dictonary of locations where
        key = location name and value = (lat, long)
        returns tuple of (closest_location , distance) """
    closest_location = None
    for city in location_dict.keys():
        distance = vincenty(location, location_dict[city]).kilometers
        if closest_location is None:
            closest_location = (city, distance)
        elif distance < closest_location[1]:
            closest_location = (city, distance)
    return closest_location

In [ ]:
train_clus['close_station'] = train_clus.apply(lambda x: closest_point((x['Latitude'],x['Longitude']), statiun_coords), axis = 1)

train_clus['close_station_name'] = [x[0] for x in train_clus['close_station'].values]
train_clus['close_station_dist'] = [x[1] for x in train_clus['close_station'].values]


train['close_station'] = train.apply(lambda x: closest_point((x['Latitude'],x['Longitude']), statiun_coords), axis = 1)
train['close_station_name'] = [x[0] for x in train['close_station'].values]
train['close_station_dist'] = [x[1] for x in train['close_station'].values]

## Visualisation jaringan station seismic dengan titik gempa 

In [ ]:
train.head()

In [ ]:
# graph of vectors connecting points to their nearest city

train.plot(kind='scatter', x='Longitude', y='Latitude',  marker='^', alpha=1, c = '#1f77b4', figsize= (30, 10 )
				)

for line in train.iterrows():
	dat = line[1]
	x1 = dat['Longitude']
	y1 = dat['Latitude']
	p2 = statiun_coords[dat['close_station_name']]
	x2 = p2[1]
	y2 = p2[0]
	plt.plot([x1,x2],[y1, y2], 'k-',linewidth=0.5)
    
    
#plt.imshow(peta_gempa,  extent=[ 93 , 118.5, -13.3 , 7.7], alpha=0.9)
#plt.imshow(peta_gempa,  extent=[ 90 , 115.5, -14.7 , 7.9], alpha=1)
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)
plt.title("Map Jaringan Lokasi Stasiun dengan Gempa", fontsize = 20)
plt.show()

In [ ]:
train5 = train[train['Magnitude'] >= 5.5]

In [ ]:
# graph of vectors connecting points to their nearest city

train5.plot(kind='scatter', x='Longitude', y='Latitude',  marker='^', alpha=1, c = '#1f77b4', figsize = (30, 10)
				)

for line in train5.iterrows():
    
	dat = line[1]
	x1 = dat['Longitude']
	y1 = dat['Latitude']
	p2 = statiun_coords[dat['close_station_name']]
	x2 = p2[1]
	y2 = p2[0]
    
	plt.plot([x1,x2],[y1, y2], 'k-',linewidth=0.5)
    
    
#plt.imshow(peta_gempa,  extent=[ 93 , 118.5, -13.3 , 7.7], alpha=0.9)
#plt.imshow(peta_gempa,  extent=[ 90 , 115.5, -14.7 , 7.9], alpha=1)
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)
plt.title("Map Jaringan Lokasi Stasiun dengan Gempa magnitude di atas 5.5 SR", fontsize = 20)
plt.show()